In [13]:
import json


class Judger:
    # Initialize Judger, with the path of tag list
    def __init__(self, tag_path):
        self.tag_dic = {}
        f = open(tag_path, "r", encoding='utf-8')
        self.task_cnt = 0
        for line in f:
            self.task_cnt += 1
            self.tag_dic[line[:-1]] = self.task_cnt

    # Format the result generated by the Predictor class
    @staticmethod
    def format_result(result):
        rex = {"tags": []}
        res_art = []
        for x in result["tags"]:
            if not (x is None):
                res_art.append(int(x))
        rex["tags"] = res_art

        return rex

    # Gen new results according to the truth and users output
    def gen_new_result(self, result, truth, label):

        s1 = set()
        for tag in label:
            s1.add(self.tag_dic[tag.replace(' ', '')])
        s2 = set()
        for name in truth:
            s2.add(self.tag_dic[name.replace(' ', '')])

        for a in range(0, self.task_cnt):
            in1 = (a + 1) in s1
            in2 = (a + 1) in s2
            if in1:
                if in2:
                    result[0][a]["TP"] += 1
                else:
                    result[0][a]["FP"] += 1
            else:
                if in2:
                    result[0][a]["FN"] += 1
                else:
                    result[0][a]["TN"] += 1

        return result

    # Calculate precision, recall and f1 value
    # According to https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
    @staticmethod
    def get_value(res):
        if res["TP"] == 0:
            if res["FP"] == 0 and res["FN"] == 0:
                precision = 1.0
                recall = 1.0
                f1 = 1.0
            else:
                precision = 0.0
                recall = 0.0
                f1 = 0.0
        else:
            precision = 1.0 * res["TP"] / (res["TP"] + res["FP"])
            recall = 1.0 * res["TP"] / (res["TP"] + res["FN"])
            f1 = 2 * precision * recall / (precision + recall)

        return precision, recall, f1

    # Generate score
    def gen_score(self, arr):
        sumf = 0
        y = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}
        for x in arr[0]:
            p, r, f = self.get_value(x)
            sumf += f
            for z in x.keys():
                y[z] += x[z]

        _, __, f_ = self.get_value(y)

        return (f_ + sumf * 1.0 / len(arr[0])) / 2.0

    # Test with ground truth path and the user's output path
    def test(self, truth_path, output_path):
        cnt = 0
        result = [[]]
        for a in range(0, self.task_cnt):
            result[0].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})

        with open(truth_path, "r", encoding='utf-8') as inf, open(output_path, "r", encoding='utf-8') as ouf:
            for line in inf:
                ground_doc = json.loads(line)
                user_doc = json.loads(ouf.readline())
                for ind in range(len(ground_doc)):
                    ground_truth = ground_doc[ind]['labels']
                    user_output = user_doc[ind]['labels']
                    cnt += 1
                    result = self.gen_new_result(result, ground_truth, user_output)

        return result


# Generatue final_score
def get_score(truth_path_labor, output_path_labor, tag_path_labor,
              truth_path_divorce, output_path_divorce, tag_path_divorce,
              truth_path_loan, output_path_loan, tag_path_loan):
    judger_labor = Judger(tag_path=tag_path_labor)
    reslt_labor = judger_labor.test(truth_path=truth_path_labor,
                                    output_path=output_path_labor)
    score_labor = judger_labor.gen_score(reslt_labor)
    print('score_labor', score_labor)

    judger_divorce = Judger(tag_path=tag_path_divorce)
    reslt_divorce = judger_divorce.test(truth_path=truth_path_divorce,
                                        output_path=output_path_divorce)
    score_divorce = judger_divorce.gen_score(reslt_divorce)
    print('score_divorce', score_divorce)

    judger_loan = Judger(tag_path=tag_path_loan)
    reslt_loan = judger_loan.test(truth_path=truth_path_loan,
                                  output_path=output_path_loan)
    score_loan = judger_loan.gen_score(reslt_loan)
    print('score_loan', score_loan)
    
    return (score_labor + score_divorce + score_loan) / 3.0




In [14]:
if __name__ == '__main__':
    final_score = get_score(
                            truth_path_labor='labor/data_small_selected.json',
                            output_path_labor='output/labor/output.json',
                            tag_path_labor='labor/tags.txt',
                            truth_path_divorce='divorce/data_small_selected.json',
                            output_path_divorce='output/divorce/output.json',
                            tag_path_divorce='divorce/tags.txt',
                            truth_path_loan='loan/data_small_selected.json',
                            output_path_loan='output/loan/output.json',
                            tag_path_loan='loan/tags.txt')
    print(final_score)


score_labor 0.0
score_divorce 0.002391185101511123
score_loan 0.0
0.0007970617005037076


In [16]:
if __name__ == '__main__':
    final_score = get_score(
                            truth_path_labor='labor/data_small_selected.json',
                            output_path_labor='output/ClassifierChain/labor/output.json',
                            tag_path_labor='labor/tags.txt',
                            truth_path_divorce='divorce/data_small_selected.json',
                            output_path_divorce='output/ClassifierChain/divorce/output.json',
                            tag_path_divorce='divorce/tags.txt',
                            truth_path_loan='loan/data_small_selected.json',
                            output_path_loan='output/ClassifierChain/loan/output.json',
                            tag_path_loan='loan/tags.txt')
    print(final_score)


score_labor 0.03426911596830585
score_divorce 0.05159578736899335
score_loan 0.03806323717853543
0.041309380171944875
